## VQE
Tenemos 
$f(x_0, x_1,x_2) = 2x_0 - 4 x_0 x_1 + 3 x_1 x_2$

Encontrar los valores binarios de las equis que minimice la función, el valor de mínima energía.

| $x_0$ |  $x_1$ |  $x_2$ |  $f(x_{0,1,2})$ |
|-------|--------|--------|-----------------|
|0|0|0|0|
|0|0|1|0|
|0|1|0|0|
|0|1|1|3|
|1|0|0|2|
|1|0|1|2|
|1|1|0|**-2**|
|1|1|1|1|
 
Cambiamos $f(x)$ por $g(z)$ donde z sean coeficientes binarios pero de valor 1 y -1. Se logra con la conversión $x_i = (1-z_i) /2$
 
Esto que se obtiene representa un circuito cuántico donde los subíndices de z definen el canal donde se hará la operación z, esto es:

$
  g(z_0, z_1, z_2) = \frac{3}{4} + \frac{1}{4}z_1 - \frac{3}{4}z_2 - z_0 z_1 + \frac{3}{4}z_1 z_2 
$

In [3]:
import pennylane as qml
from pennylane import numpy as np

variables = 3
dev = qml.device("default.qubit", wires= variables, shots = 1000)

#@qml.template
def phi(w):
    qml.RX(w[0], wires=0)
    qml.RX(w[1], wires=1)
    qml.RX(w[2], wires=2)

@qml.qnode(dev)
def circ1(w):
    phi(w)
    return qml.expval(qml.Identity(wires=0) @ qml.Identity(wires=1) @ qml.Identity(wires=2))

@qml.qnode(dev)
def circ2(w):
    phi(w)
    return qml.expval(qml.Identity(wires=0) @ qml.PauliZ(wires=1) @ qml.Identity(wires=2))

@qml.qnode(dev)
def circ3(w):
    phi(w)
    return qml.expval(qml.Identity(wires=0) @ qml.Identity(wires=1) @ qml.PauliZ(wires=2))

@qml.qnode(dev)
def circ4(w):
    phi(w)
    return qml.expval(qml.PauliZ(wires=0) @ qml.PauliZ(wires=1) @ qml.Identity(wires=2))

@qml.qnode(dev)
def circ5(w):
    phi(w)
    return qml.expval(qml.Identity(wires=0) @ qml.PauliZ(wires=1) @ qml.PauliZ(wires=2))

def value(w):
    return 3*circ1(w)/4 + circ2(w)/4 - 3*circ3(w)/4 - circ4(w) + 3*circ5(w)/4

w = qml.numpy.random.rand(3) * 2 * 3.14

gradient_fn_w = qml.grad(value, argnum=0)

lr=0.005
for epoch in range(501):
    w = w -lr*gradient_fn_w(w)
    if epoch % 50 == 0:
        print("epoch", epoch)
        print("loss", value(w))

@qml.qnode(qml.device('default.qubit', wires = variables, shots =1))
def solucion(w):
    phi(w)
    return [qml.sample(qml.PauliZ(i)) for i in range(3)]

solucion(w)

epoch 0
loss 1.2200000000000002
epoch 50
loss 0.7494999999999999
epoch 100
loss 0.49400000000000016
epoch 150
loss 0.3205000000000002
epoch 200
loss 0.20900000000000007
epoch 250
loss 0.10399999999999998
epoch 300
loss 0.01749999999999985
epoch 350
loss 0.062000000000000055
epoch 400
loss 0.02300000000000002
epoch 450
loss 0.019500000000000073
epoch 500
loss -0.0044999999999999485


[array(1, dtype=int64), array(1, dtype=int64), array(-1, dtype=int64)]